# Display charts about the geographic distribution of COVID-19 cases worldwide


> ECDC Dashboard : http://bit.ly/38Ysgum

> Data source is "today’s data on the geographic distribution of COVID-19 cases worldwide": http://bit.ly/2U9YcGS

In [1]:
# setup the python program
import pandas as pd
import numpy as np
from plotly.offline import iplot
import plotly.graph_objs as go
from datetime import datetime, timedelta
import locale
import urllib.request, urllib.error
import ipywidgets as widgets
locale.setlocale(locale.LC_ALL, '')

'en_US.UTF-8'

# Load the LATEST data from the European Center for Disease Prevention and Control (ECDC)

In [0]:
def check_if_data_source_exists(url):
  """
  check if an url exists
  """
  url_exists = False
  try:
    conn = urllib.request.urlopen(url)
  except urllib.error.HTTPError as e:
      # Return code error (e.g. 404, 501, ...)
      # ...
      print('HTTPError: {}'.format(e.code))
      url_exists = False
  except urllib.error.URLError as e:
      # Not an HTTP-specific error (e.g. connection refused)
      # ...
      print('URLError: {}'.format(e.reason))
      url_exists = False
  else:
      # 200
      # ...
      print('The url exists')
      url_exists = True
  return url_exists

In [0]:
def find_latest_datasource(url):
  """
  find the latest datasource starting today from the ECDC website

  format of the url: 
    https://www.ecdc.europa.eu/sites/default/files/documents/COVID-19-geographic-disbtribution-worldwide-2020-03-17.xlsx
    https://www.ecdc.europa.eu/sites/default/files/documents/COVID-19-geographic-disbtribution-worldwide-%latest_datasource_date%.xlsx

  """
  is_found=False
  i=-1
  latest_datasource_date = (datetime.now() - timedelta(i)).strftime('%Y-%m-%d')
  while is_found == False and i<5:
    i += 1
    latest_datasource_date = (datetime.now() - timedelta(i)).strftime('%Y-%m-%d')
    
    datasource_url = url.replace('%latest_datasource_date%', latest_datasource_date)
    print(f'Search for this latest date: {latest_datasource_date}',
          f'datasource_url: {datasource_url}')
    is_found = check_if_data_source_exists(datasource_url)

  if is_found == True:
    print(f'Download the data until : {latest_datasource_date}')
    return datasource_url
  else:
    return None

In [4]:
# load the data
datasource_url_template = "https://www.ecdc.europa.eu/sites/default/files/documents/COVID-19-geographic-disbtribution-worldwide-%latest_datasource_date%.xls"
if datasource_url_template is None:
  print(f'There is no datasource on the ECDC website. is https://www.ecdc.europa.eu down?')
  exit
datasource_url = find_latest_datasource(datasource_url_template)

Search for this latest date: 2020-03-18 datasource_url: https://www.ecdc.europa.eu/sites/default/files/documents/COVID-19-geographic-disbtribution-worldwide-2020-03-18.xls
The url exists
Download the data until : 2020-03-18


In [0]:
df = pd.read_excel(datasource_url)

# What is the range of data that has been collected from the ECDC?

In [6]:
start_date = df.DateRep.min().strftime("%b %d %Y")
end_date = df.DateRep.max().strftime("%b %d %Y")

print(
    f'DATE RANGE:\n',
    f'start date : {start_date}\n', 
    f'end date : {end_date}'
      )

DATE RANGE:
 start date : Dec 31 2019
 end date : Mar 18 2020


# Display Totals : Confirmed Cases and Deaths

In [7]:
cv19_countries_day = df.groupby(by=['DateRep','Countries and territories']).sum()[['Cases','Deaths']]
Total_confirmed = cv19_countries_day.groupby('DateRep').sum()[['Cases','Deaths']].sum()['Cases']
Total_deaths = cv19_countries_day.groupby('DateRep').sum()[['Cases','Deaths']].sum()['Deaths']
print(f"{end_date}:\n Total Confirmed Cases: {Total_confirmed:n}\n Total Deaths: {Total_deaths:n}")

Mar 18 2020:
 Total Confirmed Cases: 194,909
 Total Deaths: 7,876


# Group the data by daily-New Confirmed Cases (WW)

In [8]:
cv19_countries_day_new_confirmed_cases = df.groupby(by=['DateRep'], as_index=False)['Cases'].sum()
print(f'WW New Confirmed cases first-5 records:\n\n {cv19_countries_day_new_confirmed_cases.head(5)}')
print(f'\nWW New Confirmed cases last-5 records:\n\n {cv19_countries_day_new_confirmed_cases.tail(5)}')

WW New Confirmed cases first-5 records:

      DateRep  Cases
0 2019-12-31     27
1 2020-01-01      0
2 2020-01-02      0
3 2020-01-03     17
4 2020-01-04      0

WW New Confirmed cases last-5 records:

       DateRep  Cases
74 2020-03-14   9375
75 2020-03-15   8140
76 2020-03-16  16051
77 2020-03-17  12745
78 2020-03-18  14750


# Group the data by daily Deaths (WW)

In [9]:
cv19_countries_day_new_deaths = df.groupby(by=['DateRep'], as_index=False)['Deaths'].sum()
print(f'WW New Deaths first-5 records:\n\n {cv19_countries_day_new_deaths.head(5)}')
print(f'nWW New Deaths last-5 records:\n\n {cv19_countries_day_new_deaths.tail(5)}')

WW New Deaths first-5 records:

      DateRep  Deaths
0 2019-12-31       0
1 2020-01-01       0
2 2020-01-02       0
3 2020-01-03       0
4 2020-01-04       0
nWW New Deaths last-5 records:

       DateRep  Deaths
74 2020-03-14     438
75 2020-03-15     354
76 2020-03-16     746
77 2020-03-17     596
78 2020-03-18     773


# Draw a chart with daily New (WW) Confirmed Cases and Deaths

In [10]:
data = [
        go.Scatter(
        name="WW New Confirmed cases",
        x=cv19_countries_day_new_confirmed_cases['DateRep'],
        y=cv19_countries_day_new_confirmed_cases['Cases']
        ),
        go.Scatter(
        name="WW New Deaths",
        x=cv19_countries_day_new_deaths['DateRep'],
        y=cv19_countries_day_new_deaths['Deaths']
        ),
]

# plot titles and axis labels
layout = go.Layout(
    title=f'Worldwide New Confirmed cases and Deaths',
    xaxis=dict(
        title='Date'
    ),
    yaxis=dict(
        title='New Confirmed cases/New Deaths'
    )
)

# plot the Figure object using the 'iplot' method
fig = go.Figure(data=data, layout=layout)
iplot(fig)


# ACTION : Choose the countries you want to compare or update the list of countries you want to study

In [18]:
countries = ['China', 'Belgium', 'Spain', 'France', 'Italy', 'United_States_of_America']
countries_widget = widgets.SelectMultiple(
    options=df['Countries and territories'].unique().tolist(),
    value=countries,
    rows=10,
    description='Countries and territories',
    disabled=False
)

try:
  display(countries_widget)
  #print('you are running a jupyter notebook')
except:
  print('you are not running a jupyter notebook')

SelectMultiple(description='Countries and territories', index=(31, 15, 128, 50, 70, 144), options=('Afghanista…

In [12]:
if countries_widget is not None:
    if countries_widget.value is not None:
      countries = countries_widget.value
      
print(f'Selected countries : {countries}')

Selected countries : ('China', 'Belgium', 'Spain', 'France', 'Italy', 'United_States_of_America')


In [0]:
def display_bar_chart(countries, 
                      bar_name="New confirmed cases in", 
                      bar_x='DateRep', 
                      bar_y='Cases',
                      layout_x_title='Date', 
                      layout_y_title='New confirmed cases'
                      ):
  # use the DataFrame columns for generating data

  data = []
  for country in countries:
    # print(f"{country}")
    data.append(
        go.Bar(
          name=f"{bar_name} {country}",
          x=df[df['Countries and territories']==country][bar_x],
          y=df[df['Countries and territories']==country][bar_y]
        )
      )
  # plot titles and axis labels
  layout = go.Layout(
      title=f'{bar_name} {countries}',
      xaxis=dict(
          title=layout_x_title
      ),
      yaxis=dict(
          title=layout_y_title
      )
  )

  # plot the Figure object using the 'iplot' method
  fig = go.Figure(data=data, layout=layout)
  iplot(fig)

# Draw a chart including New Confirmed Cases in Italy and Belgium

## Some hints regarding the dynamic charts

> functions are available on the top-left of the screen. Click on the buttons

> the user can zoom in/zoom out

> click on a legend e.g., new confirmed cases (on the right) and focus on one single dataset

> click-and-drag your mouse to zoom on a dedicated area

> double click on the chart to reset the zoom

In [14]:
display_bar_chart(
    countries, 
    bar_name="New confirmed cases in", 
    bar_x='DateRep', 
    bar_y='Cases',
    layout_x_title='Date', 
    layout_y_title='New confirmed cases'
    )
display_bar_chart(
    countries,
    bar_name="New deaths in", 
    bar_x='DateRep', 
    bar_y='Deaths',
    layout_x_title='Date', 
    layout_y_title='New deaths'
    )

# Are they Correlations between the fields 'Deaths' and 'New cases'?

In [0]:
def display_correlation(df, selected_citizenship= 'Belgian', method='pearson'):
  correlation = df['Cases'].corr(df['Deaths'], method=method)
  print(f"correlation using '{method}'' method between '{selected_citizenship}' 'New cases' and 'Deaths': {correlation}")

In [16]:
for country in countries:
  display_correlation(df[df['Countries and territories']==country], country, method='pearson')


correlation using 'pearson'' method between 'China' 'New cases' and 'Deaths': 0.6967850705676589
correlation using 'pearson'' method between 'Belgium' 'New cases' and 'Deaths': 0.3122780416204782
correlation using 'pearson'' method between 'Spain' 'New cases' and 'Deaths': 0.8413012488777472
correlation using 'pearson'' method between 'France' 'New cases' and 'Deaths': 0.9351254311146221
correlation using 'pearson'' method between 'Italy' 'New cases' and 'Deaths': 0.9344236685737289
correlation using 'pearson'' method between 'United_States_of_America' 'New cases' and 'Deaths': 0.9615831512522206


# CHANGELOG

v.0.0.2

> fix: ECDC has changed the format of the file

  > filename is renamed *.xlsx into *.xls

> feat: add widget to select the countries you want a analysis about

> refactor: one function displays diverse bart charts

v.0.0.1 
  
> fix: ECDC has changed the format of the file

    > filename is renamed *.xls into *.xlsx
  
    > renamed fields: CountryExp -> Countries and territories, NewConfCases -> Cases, Newdeaths -> Deaths
  
    > removed fields: Gaul1Nuts1, EU

v.0.0.0 

    > feat: display charts for the World, Belgium and Italy

# LICENSE

(c) 2010-2020 Abdelkrim Boujraf, alt-f1 sprl <http://www.alt-f1.be>, Abdelkrim Boujraf <http://www.alt-f1.be/literature.html>

This work is licensed under a Creative Commons Attribution 4.0 International License: <http://creativecommons.org/licenses/by/4.0>

* Share — copy and redistribute the material in any medium or format 
* Adapt — remix, transform, and build upon the material for any purpose, even commercially

Vous êtes autorisé à <https://creativecommons.org/licenses/by/4.0/deed.fr>: 

* Partager : copier, distribuer et communiquer le matériel par tous moyens et sous tous formats
* Adapter : remixer, transformer et créer à partir du matériel pour toute utilisation, y compris commerciale